In [20]:
import numpy as np
import random
import copy
from math import e

In [21]:
def procesar_entrada(direccion_arcivo : str):
    with open (direccion_arcivo) as entrada:
        datos = entrada.read()
    datos=datos.split('\n')
    temp_inicial,temp_final =[float(_) for _ in datos[0].split(' ')]
    objetos = int(datos[1])
    capacidad = int(datos[2])
    p_w= []
    for i in range(3,len(datos)):
        p_w.insert(i, [int(datos[i].split(' ')[0]),int(datos[i].split(' ')[1])] ) 
    return(temp_inicial,temp_final,objetos,capacidad,p_w)

In [22]:
def peso(solucion):
    """
    obtiene el peso de una solucion
    """
    global p_w
    peso = sum([ p_w[i][1] for i in range(len(solucion)) if solucion[i] == 1 ])
    return(peso)

In [23]:
def valor(solucion):
  """
  obtiene el valor de una solucion
  """
  global p_w
  valor = sum([ p_w[i][0] for i in range(len(solucion)) if solucion[i] == 1 ])
  return(valor)

In [24]:
def inicial(N_objetos : int, capacidad : int ,p_w ):
  """
  crea la solucion inicial, agregando objetos aleatoreamente mientras el peso no supere a la capacidad,
  se detiene cuando el peso es igual a la capacidad, o cuando no puede meter mas elementos, ya que al
  intentar meterlos se excede la capacidad.
  """
  solucion = [0 for i in range(N_objetos)]
  posibles_objetos=[i for i in range(N_objetos)] #indices e los objetos que se pueden meter a la mochila
  peso_m = peso(solucion)
  while peso_m < capacidad:
      #print(posibles_objetos)
      if posibles_objetos == []:
          return solucion
          break
      else:
          meter_objeto = random.choice(posibles_objetos)
          solucion[meter_objeto] = 1
          #print(solucion,'despues intentar meter')
          if peso(solucion) > capacidad:
              solucion[meter_objeto] = 0
                # si al meter un objeto, el peso es mayor a la capacidad, lo quitamos para no volver a escogerlo edspues
          else:
              peso_m  = peso(solucion)
          posibles_objetos.remove(meter_objeto)
  return solucion

In [25]:
def vecindario(solucion):
  """
  crea el vecindario de una solucion, solo se toman las soluciones validas, despues devuelve una solucion al azar
  """
  global capacidad
  vecindario = []
  for i in range(len(solucion)) :
      vecino = copy.deepcopy(solucion)
      if vecino[i] == 0:
          vecino[i] = 1
          if peso(vecino) < capacidad:
              vecindario.append(vecino)
      else:
          vecino[i] = 0
          if peso(vecino) < capacidad:
              vecindario.append(vecino)
  return(random.choice(vecindario)) #regresamos un vecino al azar como dice el manual

In [26]:
def objetos_en_mochila(solucion):
  """ indica cuales objetos estan en lamochina, consierando una solucion """
  return([ i for i in range(len(solucion)) if (solucion[i] == 1)])

In [27]:
def imprimirResultados(resultados,m):
    ordenados = sorted(resultados, key = lambda x : x[1], reverse=True)
    media = np.mean([ordenados[i][1] for i in range(m)])
    medianas = []
    desviacion = np.std([ordenados[i][1] for i in range(m)])
    print('Mejor solucion encontrada')
    print(objetos_en_mochila(ordenados[0][0]),ordenados[0][0])
    print('{} {} \n{} {}'.format('Con un valor de',ordenados[0][1],
                             'Y un peso de',ordenados[0][2] ),'\n' )
    
    print('Peor solución encontrada')
    print(objetos_en_mochila(ordenados[-1][0]),ordenados[-1][0])
    print('{} {} \n{} {}'.format('Con un valor de',ordenados[-1][1],
                                 'Y un peso de',ordenados[-1][2]
                                ),'\n')
    if m %2 == 0:
        medianas.append(ordenados[int(m/2)])
        medianas.append(ordenados[int((m/2)-1)])
        if medianas[0] == medianas[1]: #verificamos que los elementos de la mediana no sean el mismo
            medianas.pop()
            print('Mediana')
            print(objetos_en_mochila(medianas[0][0]),medianas[0][0])
            print('{} {} \n{} {}'.format('Con un valor de',medianas[0][1],
                                 'Y un peso de',medianas[0][2]
                                ),'\n')
        else:
            print('{} \n{} {} y {} {}'.format('Medianas',objetos_en_mochila(medianas[0][0]),medianas[0][0],
                                                      objetos_en_mochila(medianas[0][0]),medianas[1][0] ))
            print('{} {} \n{} {} y {}'.format('Con un valor de',medianas[0][1],
                                 'Y un pesos de',medianas[0][2],medianas[1][2]
                                ))                     
    else:
        medianas.append(ordenados[int(m//2)])
        print('Mediana')
        print(objetos_en_mochila(medianas[0][0]), medianas[0][0])
        print('{} {} \n{} {}'.format('Con un valor de',medianas[0][1],
                                 'Y un peso de',medianas[0][2]
                                ))
    print('Media del valor de las soluciones encontradas',media,'\n')
    print('Desviacion estandar de los resultados', desviacion)

In [28]:
def recocido(entrada : dir, m : int):
    global p_w, capacidad
    soluciones = []
    for i in range(m):
        t_inicial,t_final,N_objetos,capacidad,p_w = procesar_entrada(entrada)
        s_inicial = inicial(N_objetos,capacidad, p_w)
        x_t = s_inicial
        valor_x_t=valor(x_t)
        mejor_valor = valor(s_inicial)
        mejor_s = s_inicial
        peso_mejor_s = peso(mejor_s)
        t_actual = t_inicial
        while t_actual >= t_final:
            y = vecindario(x_t)
            y_valor = valor(y)
            if y_valor > mejor_valor:
                mejor_valor  = y_valor
                mejor_s = y
                peso_mejor_s = peso(y)
            else:
                if y_valor >= valor_x_t or random.random() < e**( (-(valor_x_t - y_valor)/t_actual)   ):
                    x_t = y
                    valor_x_t = y_valor 
            t_actual = t_actual * 0.99
        soluciones.append([mejor_s,mejor_valor,peso_mejor_s])
    imprimirResultados(soluciones,m)

In [30]:
recocido('recocido.txt',100)

Mejor solucion encontrada
[0, 1, 4] [1, 1, 0, 0, 1]
Con un valor de 42 
Y un peso de 15 

Peor solución encontrada
[1, 4] [0, 1, 0, 0, 1]
Con un valor de 37 
Y un peso de 13 

Mediana
[1, 4] [0, 1, 0, 0, 1]
Con un valor de 37 
Y un peso de 13 

Media del valor de las soluciones encontradas 38.7 

Desviacion estandar de los resultados 2.3685438564654016
